In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import xgboost as xgb


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
members = pd.read_csv("members.csv",parse_dates=['registration_init_time','expiration_date'], dtype={'city': np.int8, 'bd': np.int16, 'registered_via': np.int8})
train = pd.read_csv('train.csv', dtype={'is_churn' : np.int8})
transactions = pd.read_csv('transactions.csv', parse_dates=['transaction_date','membership_expire_date'], dtype={'payment_method_id': np.int8, 'payment_plan_days': np.int16, 'plan_list_price': np.int16, 'actual_amount_paid': np.int16, 'is_auto_renew': np.int8, 'is_cancel': np.int8})

In [3]:
test = pd.read_csv("sample_submission_zero.csv")

In [4]:
#map info to each user in the train set
df_train = pd.merge(train,members,how='left', on="msno")
df_train.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,expiration_date
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,female,9.0,2005-04-06,2017-09-07
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,male,9.0,2005-04-07,2017-03-21
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,female,9.0,2005-10-16,2017-02-03
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,female,9.0,2005-11-02,2017-09-26
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,male,9.0,2005-12-28,2017-09-27


In [5]:
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,2015-09-30,2015-11-01,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,2015-09-30,2015-10-31,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,2015-09-30,2016-04-27,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,2015-09-30,2015-11-28,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,2015-09-30,2015-11-21,0


In [6]:
#get those with balance remaining
transactions["paid_diff"] = transactions["plan_list_price"]-transactions["actual_amount_paid"]
transactions.head(10)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,paid_diff
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,2015-09-30,2015-11-01,0,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,2015-09-30,2015-10-31,0,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,2015-09-30,2016-04-27,0,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,2015-09-30,2015-11-28,0,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,2015-09-30,2015-11-21,0,0
5,KN7I82kjY0Tn76Ny95ncqBUVbO7b8AXrOPqZutKpxIM=,21,30,149,149,1,2015-09-30,2015-11-07,0,0
6,m5ptKif9BjdUghHXXomSezy5ohJiHm85PE13f/3kQaw=,39,30,149,149,1,2015-09-30,2015-11-28,0,0
7,uQxbyACsPOEkTIrv9jZgoGXelGBW81ZsSZKy9fhj5Z8=,39,30,149,149,1,2015-09-30,2015-11-25,0,0
8,LUPRfoE2r3WwVWhYO/TqQhjrL/qP6CO+/ORUlr7yNc0=,39,30,149,149,1,2015-09-30,2015-12-22,0,0
9,pMVjPLgVknaJYm9L0xUwxA2jmhpK7gfhGnsmmwX1mrE=,39,30,149,149,1,2015-09-30,2015-11-18,0,0


In [7]:
#aggregate information per transaction
#payment_method_id first,is_auto_renew,is_cancel ->most used
#msno->count
#payment_plan_days,plan_list_price,actual_amount_paid,paid_diff -> sum
#transaction_date->min
#membership_expire_date->max
transactions_per_user = transactions.groupby(["msno"]).agg({"payment_method_id":lambda x:x.value_counts().index[0] ,"msno":'count',"payment_plan_days":"sum","plan_list_price":"sum","actual_amount_paid":"sum","is_auto_renew":lambda x:x.value_counts().index[0],"transaction_date":"min","membership_expire_date":"max","is_cancel":lambda x:x.value_counts().index[0],"paid_diff":"sum"})
transactions_per_user.head()

,paid_diff,actual_amount_paid,transaction_date,plan_list_price,is_auto_renew,is_cancel,msno,payment_method_id,membership_expire_date,payment_plan_days
msno,,,,,,,,,,
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0,0,2016-09-09,0,0,0,1,35,2016-09-14,7
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0,1788,2015-11-21,1788,0,0,1,38,2017-01-04,410
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,396,2016-11-16,396,1,0,4,41,2017-03-15,120
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,-149,2831,2015-01-31,2682,1,0,19,39,2017-03-19,543
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,-149,3874,2015-01-26,3725,1,0,26,41,2017-03-26,750


In [8]:
#get how long the user has subscribed
transactions_per_user["duration"]=transactions_per_user["membership_expire_date"]-transactions_per_user["transaction_date"]
transactions_per_user.duration=transactions_per_user.duration.dt.days
transactions_per_user.head()

,paid_diff,actual_amount_paid,transaction_date,plan_list_price,is_auto_renew,is_cancel,msno,payment_method_id,membership_expire_date,payment_plan_days,duration
msno,,,,,,,,,,,
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0,0,2016-09-09,0,0,0,1,35,2016-09-14,7,5
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0,1788,2015-11-21,1788,0,0,1,38,2017-01-04,410,410
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,396,2016-11-16,396,1,0,4,41,2017-03-15,120,119
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,-149,2831,2015-01-31,2682,1,0,19,39,2017-03-19,543,778
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,-149,3874,2015-01-26,3725,1,0,26,41,2017-03-26,750,790


In [ ]:
transactions_per_user["msno_count"]=transactions_per_user["msno"]

In [9]:
payment_method_dummies = pd.get_dummies(transactions_per_user["payment_method_id"],prefix="method")
transactions_per_user.drop(["payment_method_id"],axis=1,inplace=True)
transactions_per_user=transactions_per_user.join(payment_method_dummies)
transactions_per_user.head()

,paid_diff,actual_amount_paid,transaction_date,plan_list_price,is_auto_renew,is_cancel,msno,membership_expire_date,payment_plan_days,duration,...,method_32,method_33,method_34,method_35,method_36,method_37,method_38,method_39,method_40,method_41
msno,,,,,,,,,,,,,,,,,,,,,
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0,0,2016-09-09,0,0,0,1,2016-09-14,7,5,...,0,0,0,1,0,0,0,0,0,0
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0,1788,2015-11-21,1788,0,0,1,2017-01-04,410,410,...,0,0,0,0,0,0,1,0,0,0
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,396,2016-11-16,396,1,0,4,2017-03-15,120,119,...,0,0,0,0,0,0,0,0,0,1
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,-149,2831,2015-01-31,2682,1,0,19,2017-03-19,543,778,...,0,0,0,0,0,0,0,1,0,0
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,-149,3874,2015-01-26,3725,1,0,26,2017-03-26,750,790,...,0,0,0,0,0,0,0,0,0,1


In [10]:
#delete dates and msno column
del transactions_per_user["msno"]
del transactions_per_user["transaction_date"]
del transactions_per_user["membership_expire_date"]
transactions_per_user.head()

,paid_diff,actual_amount_paid,plan_list_price,is_auto_renew,is_cancel,payment_plan_days,duration,method_1,method_2,method_3,...,method_32,method_33,method_34,method_35,method_36,method_37,method_38,method_39,method_40,method_41
msno,,,,,,,,,,,,,,,,,,,,,
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0,0,0,0,0,7,5,0,0,0,...,0,0,0,1,0,0,0,0,0,0
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0,1788,1788,0,0,410,410,0,0,0,...,0,0,0,0,0,0,1,0,0,0
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,396,396,1,0,120,119,0,0,0,...,0,0,0,0,0,0,0,0,0,1
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,-149,2831,2682,1,0,543,778,0,0,0,...,0,0,0,0,0,0,0,1,0,0
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,-149,3874,3725,1,0,750,790,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
transactions_per_user.to_csv("transactions_summary.csv")

In [12]:
df_train.set_index(["msno"],inplace=True)

In [13]:
#map info to each user in the train set
df_train=df_train.join(transactions_per_user)
df_train.head()

,is_churn,city,bd,gender,registered_via,registration_init_time,expiration_date,paid_diff,actual_amount_paid,plan_list_price,...,method_32,method_33,method_34,method_35,method_36,method_37,method_38,method_39,method_40,method_41
msno,,,,,,,,,,,,,,,,,,,,,
waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,female,9.0,2005-04-06,2017-09-07,0,149,149,...,0,0,0,0,0,0,1,0,0,0
QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,male,9.0,2005-04-07,2017-03-21,-149,3458,3309,...,0,0,0,0,0,0,0,1,0,0
fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,female,9.0,2005-10-16,2017-02-03,0,1492,1492,...,0,0,0,0,0,0,0,1,0,0
mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,female,9.0,2005-11-02,2017-09-26,0,1788,1788,...,0,0,0,0,0,0,1,0,0,0
XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,male,9.0,2005-12-28,2017-09-27,0,3576,3576,...,0,0,0,0,0,0,1,0,0,0


In [14]:
"""we see that there are a lot of nulls for gender, so we may opt not to include it first. We also would not need registration
and expiration date.Fill other nulls with 0
"""
del df_train["gender"]
del df_train["expiration_date"]
del df_train["registration_init_time"]
df_train.fillna(0,inplace=True)

In [15]:
#delete other dfs to free up RAM
del transactions
del train

import gc
gc.collect()

1925

In [16]:
df_train.head(20)

,is_churn,city,bd,registered_via,paid_diff,actual_amount_paid,plan_list_price,is_auto_renew,is_cancel,payment_plan_days,...,method_32,method_33,method_34,method_35,method_36,method_37,method_38,method_39,method_40,method_41
msno,,,,,,,,,,,,,,,,,,,,,
waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,9.0,0,149,149,0,0,37,...,0,0,0,0,0,0,1,0,0,0
QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,9.0,-149,3458,3309,1,0,663,...,0,0,0,0,0,0,0,1,0,0
fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,9.0,0,1492,1492,1,0,300,...,0,0,0,0,0,0,0,1,0,0
mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,9.0,0,1788,1788,0,0,820,...,0,0,0,0,0,0,1,0,0,0
XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,9.0,0,3576,3576,0,0,785,...,0,0,0,0,0,0,1,0,0,0
GBy8qSz16X5iYWD+3CMxv/Hm6OPSrXBYtmbnlRtknW0=,1,6.0,23.0,9.0,-894,3278,2384,1,0,480,...,0,1,0,0,0,0,0,0,0,0
lYLh7TdkWpIoQs3i3o6mIjLH8/IEgMWP9r7OpsLX0Vo=,1,13.0,29.0,9.0,0,1770,1770,0,0,330,...,0,0,0,0,1,0,0,0,0,0
T0FF6lumjKcqEO0O+tUH2ytc+Kb9EkeaLzcVUiTr1aE=,1,11.0,22.0,9.0,-149,3371,3222,1,0,633,...,0,0,0,0,0,0,0,0,1,0
Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,1,18.0,22.0,9.0,-150,3833,3683,0,0,720,...,0,0,0,0,1,0,0,0,0,0


In [18]:
register_via_dummies = pd.get_dummies(df_train["registered_via"],prefix="registered_id")
df_train.drop(["registered_via"],axis=1,inplace=True)
df_train=df_train.join(register_via_dummies)
df_train.head()

,is_churn,city,bd,paid_diff,actual_amount_paid,plan_list_price,is_auto_renew,is_cancel,payment_plan_days,duration,...,method_38,method_39,method_40,method_41,registered_id_0.0,registered_id_3.0,registered_id_4.0,registered_id_7.0,registered_id_9.0,registered_id_13.0
msno,,,,,,,,,,,,,,,,,,,,,
waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,0,149,149,0,0,37,98,...,1,0,0,0,0,0,0,0,1,0
QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,-149,3458,3309,1,0,663,780,...,0,1,0,0,0,0,0,0,1,0
fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,0,1492,1492,1,0,300,367,...,0,1,0,0,0,0,0,0,1,0
mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,0,1788,1788,0,0,820,413,...,1,0,0,0,0,0,0,0,1,0
XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,0,3576,3576,0,0,785,787,...,1,0,0,0,0,0,0,0,1,0


In [25]:
# Utility function to report best scores
from operator import itemgetter

def report_scores(grid_scores, n_top=10):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Neg Log Loss score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [20]:
df_class=df_train["is_churn"]
del df_train["is_churn"]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic')
param_dist = {'n_estimators': stats.randint(150, 500),
              'learning_rate': stats.uniform(0.01, 0.07),
              'subsample': stats.uniform(0.3, 0.7),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.45),
              'min_child_weight': [1, 2, 3]
             }
random_search = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 25, scoring = 'neg_log_loss', error_score = 0, verbose = 3)

random_search.fit(df_train, df_class)


In [22]:
import gc
gc.collect()


408

In [23]:
from sklearn.externals import joblib
joblib.dump(random_search.best_estimator_, 'XGBoost2.pkl')

['XGBoost2.pkl']

In [26]:
report_scores(random_search.grid_scores_)

Model with rank: 1
Neg Log Loss score: -0.113 (std: 0.000)
Parameters: {'subsample': 0.67810271267860323, 'learning_rate': 0.070050045683075587, 'n_estimators': 426, 'max_depth': 9, 'min_child_weight': 3, 'colsample_bytree': 0.81505492190234463}

Model with rank: 2
Neg Log Loss score: -0.114 (std: 0.000)
Parameters: {'subsample': 0.75887635275087595, 'learning_rate': 0.071943491910055432, 'n_estimators': 420, 'max_depth': 9, 'min_child_weight': 3, 'colsample_bytree': 0.54162623209004612}

Model with rank: 3
Neg Log Loss score: -0.117 (std: 0.001)
Parameters: {'subsample': 0.82842114311369164, 'learning_rate': 0.056982739392085917, 'n_estimators': 495, 'max_depth': 7, 'min_child_weight': 1, 'colsample_bytree': 0.72179539409980764}

Model with rank: 4
Neg Log Loss score: -0.119 (std: 0.001)
Parameters: {'subsample': 0.69646774599426386, 'learning_rate': 0.038010616652071956, 'n_estimators': 309, 'max_depth': 9, 'min_child_weight': 3, 'colsample_bytree': 0.72428197831971408}

Model with r

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [27]:
test.set_index(["msno"],inplace=True)

In [28]:
del test["is_churn"]
test=(test.join(transactions_per_user))

In [29]:
members.set_index(["msno"],inplace=True)

In [39]:
del test["city"]
del test["bd"]
test=(test.join(members))

In [40]:
del test["registration_init_time"]
del test["expiration_date"]
del test["gender"]
test.head()

,paid_diff,actual_amount_paid,plan_list_price,is_auto_renew,is_cancel,payment_plan_days,duration,method_1,method_2,method_3,...,method_35,method_36,method_37,method_38,method_39,method_40,method_41,city,bd,registered_via
msno,,,,,,,,,,,,,,,,,,,,,
ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,-596.0,894.0,298.0,1.0,0.0,60.0,730.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,28.0,3.0
f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0.0,627.0,627.0,0.0,0.0,127.0,374.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,13.0,20.0,3.0
zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0.0,2682.0,2682.0,0.0,0.0,540.0,566.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,13.0,18.0,3.0
8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,-298.0,2682.0,2384.0,1.0,0.0,487.0,790.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,7.0
K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,-427.0,2956.0,2529.0,1.0,0.0,630.0,771.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,35.0,7.0


In [41]:
register_via_dummies = pd.get_dummies(test["registered_via"],prefix="registered_id")
test.drop(["registered_via"],axis=1,inplace=True)
test=test.join(register_via_dummies)
test.head()

,paid_diff,actual_amount_paid,plan_list_price,is_auto_renew,is_cancel,payment_plan_days,duration,method_1,method_2,method_3,...,method_39,method_40,method_41,city,bd,registered_id_3.0,registered_id_4.0,registered_id_7.0,registered_id_9.0,registered_id_13.0
msno,,,,,,,,,,,,,,,,,,,,,
ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,-596.0,894.0,298.0,1.0,0.0,60.0,730.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,28.0,1,0,0,0,0
f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0.0,627.0,627.0,0.0,0.0,127.0,374.0,0.0,0.0,0.0,...,0.0,0.0,0.0,13.0,20.0,1,0,0,0,0
zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0.0,2682.0,2682.0,0.0,0.0,540.0,566.0,0.0,0.0,0.0,...,0.0,0.0,0.0,13.0,18.0,1,0,0,0,0
8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,-298.0,2682.0,2384.0,1.0,0.0,487.0,790.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0,0,1,0,0
K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,-427.0,2956.0,2529.0,1.0,0.0,630.0,771.0,0.0,0.0,0.0,...,0.0,0.0,1.0,13.0,35.0,0,0,1,0,0


In [44]:
test.fillna(0,inplace=True)
test["predict"]=random_search.best_estimator_.predict_proba(test)[:,1]

In [43]:
test["registered_id_0.0"]=0
test=test[['city', 'bd', 'paid_diff', 'actual_amount_paid', 'plan_list_price',
       'is_auto_renew', 'is_cancel', 'payment_plan_days', 'duration',
       'method_1', 'method_2', 'method_3', 'method_4', 'method_5', 'method_6',
       'method_7', 'method_8', 'method_10', 'method_11', 'method_12',
       'method_13', 'method_14', 'method_15', 'method_16', 'method_17',
       'method_18', 'method_19', 'method_20', 'method_21', 'method_22',
       'method_23', 'method_24', 'method_25', 'method_26', 'method_27',
       'method_28', 'method_29', 'method_30', 'method_31', 'method_32',
       'method_33', 'method_34', 'method_35', 'method_36', 'method_37',
       'method_38', 'method_39', 'method_40', 'method_41', 'registered_id_0.0',
       'registered_id_3.0', "registered_id_4.0",'registered_id_7.0',
       'registered_id_9.0', 'registered_id_13.0']]

In [37]:
test.head()

,paid_diff,actual_amount_paid,plan_list_price,is_auto_renew,is_cancel,payment_plan_days,duration,method_1,method_2,method_3,...,method_34,method_35,method_36,method_37,method_38,method_39,method_40,method_41,city,bd
msno,,,,,,,,,,,,,,,,,,,,,
ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,-596.0,894.0,298.0,1.0,0.0,60.0,730.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,28.0
f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0.0,627.0,627.0,0.0,0.0,127.0,374.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,13.0,20.0
zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0.0,2682.0,2682.0,0.0,0.0,540.0,566.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,13.0,18.0
8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,-298.0,2682.0,2384.0,1.0,0.0,487.0,790.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,-427.0,2956.0,2529.0,1.0,0.0,630.0,771.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,35.0


In [46]:
test["msno"]=test.index
test["is_churn"]=test.predict
submission = pd.DataFrame(index=test.index)
submission.index = range(len(submission))
submission=test[["msno","is_churn"]]
submission.index = range(len(submission))
submission

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0.036301
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0.522111
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0.107923
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,0.890792
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,0.068865
5,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,0.055905
6,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,0.000943
7,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,0.282510
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,0.351240
9,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,0.053609


In [47]:
submission.to_csv("submission4.csv",index=False)